In [ ]:
!curl -sSOL https://dlcdn.apache.org/kafka/3.5.0/kafka_2.13-3.5.0.tgz
!tar -xzf kafka_2.13-3.5.0.tgz

In [ ]:
!./kafka_2.13-3.5.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.5.0/config/zookeeper.properties
!./kafka_2.13-3.5.0/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.5.0/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [ ]:
!./kafka_2.13-3.5.0/bin/kafka-console-producer.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 2 --topic patients_json

In [1]:
!./kafka_2.13-3.5.0/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic patients_json2 < id_data_unknown.csv
!./kafka_2.13-3.5.0/bin/kafka-console-consumer.sh --topic patients_json2  --bootstrap-server localhost:9092 --from-beginning  --max-messages 5

904357,11.8,17.26,75.26,431.9,0.09087,0.06232,0.02853,0.01638,0.1847,0.06019,0.3438,1.14,2.225,25.06,0.005463,0.01964,0.02079,0.005398,0.01477,0.003071,13.45,24.49,86,562,0.1244,0.1726,0.1449,0.05356,0.2779,0.08121
90439701,17.91,21.02,124.4,994,0.123,0.2576,0.3189,0.1198,0.2113,0.07115,0.403,0.7747,3.123,41.51,0.007159,0.03718,0.06165,0.01051,0.01591,0.005099,20.8,27.78,149.6,1304,0.1873,0.5917,0.9034,0.1964,0.3245,0.1198
904647,11.93,10.91,76.14,442.7,0.08872,0.05242,0.02606,0.01796,0.1601,0.05541,0.2522,1.045,1.649,18.95,0.006175,0.01204,0.01376,0.005832,0.01096,0.001857,13.8,20.14,87.64,589.5,0.1374,0.1575,0.1514,0.06876,0.246,0.07262
904689,12.96,18.29,84.18,525.2,0.07351,0.07899,0.04057,0.01883,0.1874,0.05899,0.2357,1.299,2.397,20.21,0.003629,0.03713,0.03452,0.01065,0.02632,0.003705,14.13,24.61,96.31,621.9,0.09329,0.2318,0.1604,0.06608,0.3207,0.07247
9047,12.94,16.17,83.18,507.6,0.09879,0.08836,0.03296,0.0239,0.1735,0.062,0.1458,0.905,0.9975,11.36,0.002887,0.01285,0.01613,0.00730

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=5f85cccf7bd645d81bfccfdcf273c9e894e9426893dc721632d79f9280a2b87e
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [4]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StringType, IntegerType, TimestampType, FloatType
from pyspark.sql import functions as F
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, CountVectorizer, StringIndexer, IndexToString

spark = SparkSession.builder \
    .appName('hw_8') \
    .config('spark.executor.instances', 4) \
    .getOrCreate()

kafka_brokers = "localhost"

# загружаю свои данные
df1 = spark.read.csv('/content/id_data.csv', header=True, inferSchema=True).drop('_c0')
df2 = spark.read.csv('/content/id_diag.csv', header=True, inferSchema=True).drop('_c0')

df1.printSchema()
df2.printSchema()
)

# создаю для них представление
df1.createOrReplaceTempView("id_data")
df2.createOrReplaceTempView("id_diag")

# проверяю как данные загрузились
spark.sql("select * from id_data").show(5, False)
spark.sql("select * from id_diag").show(5, False)

# # делаю join двух таблиц, чтобы обучить модель
patients_known = spark.sql("""
select *
from id_data
join id_diag
where id_data.id = id_diag.id """)


# обучение модели
categoricalColumns = []
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + 'Index').setHandleInvalid("keep")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()],
                                     outputCols=[categoricalCol + "classVec"]).setHandleInvalid("keep")
    stages += [stringIndexer, encoder]

label_stringIdx = StringIndexer(inputCol='diagnosis', outputCol='label').setHandleInvalid("keep")
stages += [label_stringIdx]

numericCols = ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean',
               'concavity_mean',
               'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se',
               'perimeter_se',
               'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
               'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst',
               'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst',
               'fractal_dimension_worst']

assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features").setHandleInvalid("keep")
stages += [assembler]

lr = LogisticRegression(featuresCol='features', labelCol='label', maxIter=10)
stages += [lr]

label_stringIdx_fit = label_stringIdx.fit(patients_known)
indexToStringEstimator = IndexToString().setInputCol("prediction").setOutputCol("category").setLabels(
    label_stringIdx_fit.labels)

stages += [indexToStringEstimator]

pipeline = Pipeline().setStages(stages)
pipelineModel = pipeline.fit(patients_known)


# сохраняем модель

pipelineModel.write().overwrite().save("my_LR_model_patients")

# проверяем работу модели и выдаем столбцы диагноза из файла и предсказанный
pipelineModel.transform(patients_known).select("diagnosis", "category").show(5)



root
 |-- id: integer (nullable = true)
 |-- radius_mean: double (nullable = true)
 |-- texture_mean: double (nullable = true)
 |-- perimeter_mean: double (nullable = true)
 |-- area_mean: double (nullable = true)
 |-- smoothness_mean: double (nullable = true)
 |-- compactness_mean: double (nullable = true)
 |-- concavity_mean: double (nullable = true)
 |-- concave points_mean: double (nullable = true)
 |-- symmetry_mean: double (nullable = true)
 |-- fractal_dimension_mean: double (nullable = true)
 |-- radius_se: double (nullable = true)
 |-- texture_se: double (nullable = true)
 |-- perimeter_se: double (nullable = true)
 |-- area_se: double (nullable = true)
 |-- smoothness_se: double (nullable = true)
 |-- compactness_se: double (nullable = true)
 |-- concavity_se: double (nullable = true)
 |-- concave points_se: double (nullable = true)
 |-- symmetry_se: double (nullable = true)
 |-- fractal_dimension_se: double (nullable = true)
 |-- radius_worst: double (nullable = true)
 |-- t

In [3]:
import csv
import json

csvfile = open('/content/id_data_unknown.csv', 'r')
jsonfile = open('/content/id_data_unknown.json', 'w')


reader = csv.DictReader(csvfile)
for row in reader:
   json.dump(row, jsonfile)
   jsonfile.write('\n')

jsonfile = open('/content/id_data_unknown.json', 'w')

In [8]:
!./kafka_2.13-3.5.0/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic patients_json < /content/id_data_unknown.json
!./kafka_2.13-3.5.0/bin/kafka-console-consumer.sh --topic patients_json  --bootstrap-server localhost:9092 --from-beginning  --max-messages 5

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [ ]:
!apt-get install openjdk-8-jre

In [ ]:
!wget https://downloads.apache.org/cassandra/4.0.11/apache-cassandra-4.0.11-bin.tar.gz
!tar -xzvf apache-cassandra-4.0.11-bin.tar.gz

In [ ]:
!apache-cassandra-4.0.11/bin/cassandra -R

In [ ]:
!pip install cassandra-driver

In [1]:
!apache-cassandra-4.0.11/bin/cqlsh

Connected to Test Cluster at 127.0.0.1:9042
[cqlsh 6.0.0 | Cassandra 4.0.11 | CQL spec 3.4.5 | Native protocol v5]
Use HELP for help.
cqlsh> CREATE KEYSPACE IF NOT EXISTS hw_8      WITH REPLICATION = {          'class': 'SimpleStrategy',          'replication_factor': 1       };
cqlsh> CREATE TABLE IF NOT EXISTS hw_8.prediction (   id BIGINT,   "radius_mean" Float,   "texture_mean" Float,   "perimeter_mean" Float,   "area_mean" Float,   "smoothness_mean" Float,   "compactness_mean" Float,   "concavity_mean" Float,   "concave points_mean" Float,   "symmetry_mean" Float,   "fractal_dimension_mean" Float,   "radius_se" Float,   "texture_se" Float,   "perimeter_se" Float,   "area_se" Float,   "smoothness_se" Float,   "compactness_se" Float,   "concavity_se" Float,   "concave points_se" Float,   "symmetry_se" Float,   "fractal_dimension_se" Float,   "radius_worst" Float,   "texture_worst" Float,   "perimeter_worst" Float,   "area_worst" Float,   "smoothness_worst" Float,   "compactness_wors

In [ ]:
# CREATE KEYSPACE IF NOT EXISTS hw_8
#      WITH REPLICATION = {
#         'class': 'SimpleStrategy',
#          'replication_factor': 1
#      };

# CREATE TABLE IF NOT EXISTS hw_8.prediction (
#   id BIGINT,
#   "radius_mean" Float,
#   "texture_mean" Float,
#   "perimeter_mean" Float,
#   "area_mean" Float,
#   "smoothness_mean" Float,
#   "compactness_mean" Float,
#   "concavity_mean" Float,
#   "concave points_mean" Float,
#   "symmetry_mean" Float,
#   "fractal_dimension_mean" Float,
#   "radius_se" Float,
#   "texture_se" Float,
#   "perimeter_se" Float,
#   "area_se" Float,
#   "smoothness_se" Float,
#   "compactness_se" Float,
#   "concavity_se" Float,
#   "concave points_se" Float,
#   "symmetry_se" Float,
#   "fractal_dimension_se" Float,
#   "radius_worst" Float,
#   "texture_worst" Float,
#   "perimeter_worst" Float,
#   "area_worst" Float,
#   "smoothness_worst" Float,
#   "compactness_worst" Float,
#   "concavity_worst" Float,
#   "concave points_worst" Float,
#   "symmetry_worst" Float,
#   "fractal_dimension_worst" Float,
#   PRIMARY KEY (id)
# );

# SELECT keyspace_name, table_name
# FROM system_schema.tables
# WHERE keyspace_name = 'hw_8';

# exit;

In [10]:
!export SPARK_KAFKA_VERSION=0.10
!pyspark --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.kafka:kafka-clients:3.3.1,com.datastax.spark:spark-cassandra-connector_2.12:3.4.0

Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6bf0383a-ea6a-4a83-a824-e5d997fc11a5;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-cl

In [ ]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StringType, IntegerType, TimestampType, FloatType
from pyspark.ml import PipelineModel
import datetime
import shutil

kafka_brokers = "localhost:9092"

# описание данных
schema = StructType()
    .add("id", IntegerType())
    .add("radius_mean", FloatType())
    .add("texture_mean", FloatType())
    .add("perimeter_mean", FloatType())
    .add("area_mean", FloatType())
    .add("smoothness_mean", FloatType())
    .add("compactness_mean", FloatType())
    .add("concavity_mean", FloatType())
    .add("concave points_mean", FloatType())
    .add("symmetry_mean", FloatType())
    .add("fractal_dimension_mean", FloatType())
    .add("radius_se", FloatType())
    .add("texture_se", FloatType())
    .add("perimeter_se", FloatType())
    .add("area_se", FloatType())
    .add("smoothness_se", FloatType())
    .add("compactness_se", FloatType())
    .add("concavity_se", FloatType())
    .add("concave points_se", FloatType())
    .add("symmetry_se", FloatType())
    .add("fractal_dimension_se", FloatType())
    .add("radius_worst", FloatType())
    .add("texture_worst", FloatType())
    .add("perimeter_worst", FloatType())
    .add("area_worst", FloatType())
    .add("smoothness_worst", FloatType())
    .add("compactness_worst", FloatType())
    .add("concavity_worst", FloatType())
    .add("concave points_worst", FloatType())
    .add("symmetry_worst", FloatType())
    .add("fractal_dimension_worst", FloatType())

patients = spark.readStream. \
    format("kafka"). \
    option("kafka.bootstrap.servers", kafka_brokers). \
    option("subscribe", "patients_json"). \
    option("startingOffsets", "earliest"). \
    option("maxOffsetsPerTrigger", "1"). \
    load()


# читаю csv файл
raw_files = spark
    .read
    .format("csv")
    .schema(schema)
    .options(path="csv_stream", header=True)
    .load()

# убираю пробелы в названиях колонок чтобы загрузить в кассандру
cass_patients = raw_files.select("id", "radius_mean", "texture_mean",
                                 "perimeter_mean", "area_mean", "smoothness_mean", "compactness_mean",
                                 "concavity_mean", F.col("concave points_mean").alias("concave_points_mean"),
                                 "symmetry_mean", "fractal_dimension_mean", "radius_se", "texture_se",
                                 "perimeter_se", "area_se", "smoothness_se", "compactness_se", "concavity_se",
                                 F.col("concave points_se").alias("concave_points_se"), "symmetry_se",
                                 "fractal_dimension_se", "radius_worst", "texture_worst", "perimeter_worst",
                                 "area_worst", "smoothness_worst", "compactness_worst", "concavity_worst",
                                 F.col("concave points_worst").alias("concave_points_worst"), "symmetry_worst",
                                 "fractal_dimension_worst")
# записываю данные в кассандру
cass_patients.write
    .format("org.apache.spark.sql.cassandra")
    .options(table="prediction", keyspace="hw_8")
    .mode("append")
    .save()

# для проверки записи читаю данные из своей таблицы
patients_df = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="prediction", keyspace="hw_8") \
    .load()

# меняю названия колонок обратно, чтобы они совпадали со схемой
cass_patients_df = patients_df.select("id", "radius_mean", "texture_mean",
                                      "perimeter_mean", "area_mean", "smoothness_mean", "compactness_mean",
                                      "concavity_mean", F.col("concave_points_mean").alias("concave points_mean"),
                                      "symmetry_mean", "fractal_dimension_mean", "radius_se", "texture_se",
                                      "perimeter_se", "area_se", "smoothness_se", "compactness_se", "concavity_se",
                                      F.col("concave_points_se").alias("concave points_se"), "symmetry_se",
                                      "fractal_dimension_se", "radius_worst", "texture_worst", "perimeter_worst",
                                      "area_worst", "smoothness_worst", "compactness_worst", "concavity_worst",
                                      F.col("concave_points_worst").alias("concave points_worst"), "symmetry_worst",
                                      "fractal_dimension_worst")

pipeline_model = PipelineModel.load("my_LR_model_patients")


def writer_logic(df, epoch_id):
    df.persist()
    df.show()
    predict = pipeline_model.transform(df)
    predict_short = predict.select("id", F.col("category").alias("diagnosis"))
    patients_list_df = df.select("id").distinct()
    patients_list_rows = patients_list_df.collect()
    patients_list = map(lambda x: str(x.__getattr__("id")), patients_list_rows)
    where_string = " id = " + " or id = ".join(patients_list)
    print("These ids from Cassandra have dublicates in kafka:")
    dublicate_cass = cass_patients_df.where(where_string)
    dublicate = dublicate_cass.select("id")
    dublicate.show()
    print("Here is what I've got after model transformation:")
    predict_short.show()
    df.unpersist()


# связываем источник Кафки и foreachBatch функцию
stream = patients_flat \
    .writeStream \
    .trigger(processingTime='100 seconds') \
    .foreachBatch(writer_logic) \
    .option("checkpointLocation", "checkpoints/patients_unknown_checkpoint")

s = stream.start()
s.stop()